This jupyter notebook contains some gaps indicated by "*Your code*". Your tasks is to fill these gaps with appropriate python code. 

Also try to answer the following questions?
* With which measure (jaccard or cosine) do you obtain larger values? 
* Which is missing in the current system, which is usually part of real-world information retrieval systems? 




In [1]:
import csv,sys,math

In [2]:
## read idf weights from file
def read_idf_weights(idf_filename,map_idf):
    max_idf=-1
    fin=open(idf_filename)
    for line in fin:
        line=line.strip()
        i=line.find(":")
        word=line[0:i]
        idf=float(line[i+1:len(line)])
        map_idf[word]=idf
        max_idf=max(max_idf,idf)
    return max_idf


In [3]:
## use union and intersections
## s1: set of words of document 1
## s2: set of words of document 2
def calculate_jaccard(s1,s2):
   ## YOUR CODE



In [4]:
# compute the scalar product between two tfidf vectors represented in a sparse format
# map_tf1: maps a word to its term frequence in document 1
# map_tf2: maps a word to its term frequence in document 2
# map_idf: maps a word to log of its inverse document frequence
# max_idf: maximum total value of logarithm of inverse document frequence
def scalar_product(map_tf1,map_tf2,map_idf,max_idf):
    valall=0
    for key,tf1 in map_tf1.items():
        idf=map_idf.get(key) ## note idf already CONTAINS the logarithm!
        if idf==None:
            idf=max_idf
        tf2=map_tf2.get(key)
        if tf2!=None:
            valall+=# YOUR CODE multiply the two tf-idf values
    valall=math.sqrt(valall)
    return valall


In [5]:
## define in terms of scalar product
## map_tf1: maps a word in document 1 to its term frequence
## map_tf2: maps a word in document 2 to its term frequence
## map_idf: maps a word to its logarithm of inverse document frequence
## max_idf: maximum value of logarithm of inverse document frequence
def calc_cosine_measure(map_tf1,map_tf2,map_idf,max_idf):
   ## Your code -> resort to scalar product, the method above


In [6]:
## read stop words from file
## file where the stop words come from
def read_stop_words(filename):
    stop_words=[]
    with open(filename) as fin:
        for line in fin:
            line=line.strip()
            stop_words.append(line)
    return stop_words


In [7]:
### tokenize string
def tokenize(s,ssep):
    tokens=[]
    sout=""
    for c in s:
        if ssep.find(c)!=-1:
            if sout!="":
                tokens.append(sout)
                sout=""
        else:
            sout+=c
    if sout!="":
        tokens.append(sout)
        sout=""
    return tokens


In [8]:
### if key exists, increase its count by one
### otherwise set its count to 1
### ht: hashtable that mapps a word to its number of occurrences in a document (term frequence)
def putcount(ht,key):
    val=ht.get(key)
    if val==None:
        ht[key]=1
    else:
        ht[key]=val+1


In [19]:
def index_texts(input_filename,stop_word_filename):
    entries=[]
    stop_words=[]

    if stop_word_filename!=None:
        stop_words=read_stop_words(stop_word_filename)

    with open(input_filename,encoding="utf-8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';',  quotechar='\"')
        line_count = 0
        first_time=True
        for row in csv_reader:
            if first_time:
                first_time=None
                continue
            map_count={}
            words=set()

            if len(row)<2:
                continue
            description=row[1]
            title=row[0]
            tokens=tokenize(description,"!? .,;)(")
            for token in tokens:
                token=token.strip()
                if token in stop_words:
                    continue
                putcount(map_count,token)
                words.add(token)
            entry={}
            entry["title"]=title
            entry["description"]=description
            entry["words"]=words
            entry["counts"]=map_count
            entries.append(entry)
    return entries


In [20]:
## add the best n results to the list
def add_n_best(li_results,result_entry,maxlen):
    inserted=None
    for i in range(0,len(li_results),1):
        new_score=result_entry["score"]
        score_li=li_results[i]["score"]
        if new_score>score_li:
            li_results.insert(i,result_entry)
            inserted=True
            break
    if not inserted and len(li_results)<maxlen:
        li_results.append(result_entry)
        return
    if len(li_results)>maxlen:
        del(li_results[len(li_results)-1])


In [21]:
def search_best_match(search_expression,entries,search_method,map_idf,max_idf):
    li_results=[]
    tokens_search_expression=tokenize(search_expression," !()?")
    map_counts1={}
    for token in tokens_search_expression:
        putcount(map_counts1,token)

    set_search_expression=set(tokens_search_expression)
    best_score=-1
    best_entry=[]
    for entry in entries:
        words=entry["words"]
        map_counts2=entry["counts"]
        if search_method=="jaccard":
            score=calculate_jaccard(set_search_expression,words)
        else:
            score=calc_cosine_measure(map_counts1,map_counts2,map_idf,max_idf)
        new_entry={"entry": entry,"score":score}
        add_n_best(li_results,new_entry,5)
    return li_results


Let us load the idf weighting table.

In [33]:
idf_weights_filename="weights.txt"
map_idf={}
max_idf=read_idf_weights(idf_weights_filename,map_idf)
print ("max_idf: ",max_idf)
print ("map_idf: ",str(map_idf)[0:100])


max_idf:  19.2454
map_idf:  {'Abend': 6.11366, 'AbenddÃ¤mmerung': 10.7859, 'Abendesse': 9.24113, 'Abscheu': 9.52425, 'Absicht': 


Define a search method and specify the words you want to search for.

In [34]:
search_expression="Linux Software"
search_method="cosine"



In [35]:
entries=index_texts("texts.csv",None)
best_matches=search_best_match(search_expression,entries,search_method,map_idf,max_idf)
for best_match in best_matches:
    entry=best_match["entry"]
    score=best_match["score"]
    description=entry["description"]
    print (entry["title"])
    print (description[0:min(len(description),150)])
    print (score)
    print ("--------------------------")

Komponentenbasierte Entwicklung Eingebetteter Systeme (KEES)
Der ständige Preisverfall und die Leistungssteigerung der Hardwarekomponenten, vor allem der Mikroprozessoren und Speicher, erschließen ständig neue E
0.5245714166210287
--------------------------
Komponentenbasierte Entwicklung Eingebetteter Systeme (KEES)
Der ständige Preisverfall und die Leistungssteigerung der Hardwarekomponenten, vor allem der Mikroprozessoren und Speicher, erschließen ständig neue E
0.5245714166210287
--------------------------
Validierung komponentenbasierter Software für Echtzeitsysteme
Der Anteil und die Bedeutung von Software bei der Entwicklung von Echtzeitsystemen wächst sehr stark. Gleichzeitig werden die Entwicklungszyklen für S
0.5180160164935071
--------------------------
Entwicklung eines webbasierten Systems zur Erfassung und Bearbeitung von Modification Requests
Nach der Inbetriebnahme einer Software gibt es vielfältige Gründe, die zu
Problemen mit der Software bei den Anwendern führen. Der